In [17]:
import os
import collections
from operator import itemgetter
import json

import pandas as pd

root_dir = "/Users/gogrean/Documents/Insight_Fellowship/Research/Mental_Health/NHANES_Survey/"
os.chdir(root_dir + "data/")

In [18]:
with open("false_side_effects.txt", 'r') as f:
    false_side_effects = [se.strip() for se in f.readlines()]
    
with open("equivalent_side_effects.txt", 'r') as f:
    equivalent_side_effects = [tuple(eq_se.strip().split(", ")) for eq_se in f.readlines()]

In [19]:
df_fda_adverse_effects = pd.read_csv("filtered_fda_drug_reports.csv", header=0)
df_meds = pd.read_csv("brand_to_generic_drug_names.csv", header=0)

In [20]:
os.chdir(root_dir + "data/side_effects/")
for m in df_meds['Generic Name']:
    df_m = df_fda_adverse_effects[df_fda_adverse_effects['simple_name'].str.lower() == m.lower()]
    m_side_effects = [y.lower() for x in df_m['adverse_effects'].str.split("; ") for y in x]
    counter=collections.Counter(m_side_effects)
    side_effects_raw = [(eff, p/sum(counter.values())*100.) for eff, p in counter.items()]
    side_effects = [(se, p) for (se, p) in side_effects_raw 
                    if se not in false_side_effects]
    for eq_se in equivalent_side_effects:
        if set(eq_se).intersection(false_side_effects):
            continue
        to_remove = []
        temp_perc = 0
        for se in eq_se:
            if se in [s[0] for s in side_effects]:
                i = [s[0] for s in side_effects].index(se)
                temp_perc += side_effects[i][1]
                to_remove.append(side_effects[i])
        side_effects += [(eq_se[0], temp_perc)]
        for removable in to_remove:
            side_effects.remove(removable)
    side_effects.sort(key=itemgetter(1), reverse=True)
    side_effects = [(s[0].title(), s[1]) for s in side_effects]
    with open(m.lower().replace("; ", "_") + ".json", 'w') as f:
        json.dump(side_effects, f)